In [1]:
import requests
import random
import csv

In [16]:
APP_ID = '636b76a3b5bd1b0812f0f34df886fe67'
API_URL = 'https://api.worldoftanks.eu/wot/account/info/'
DATA_HEADERS = ['spotted', 'battles_on_stunning_vehicles', 'hits', 'battle_avg_xp', 'draws', 'max_xp', 'survived_battles',
                'wins', 'losses', 'capture_points', 'battles', 'damage_dealt', 'damage_received', 'max_frags', 'shots',
                'frags', 'max_damage', 'xp', 'avg_damage_assisted', 'piercings', 'user_id', 'nickname']
user_count = 5

In [3]:
def saveRecordToFile(path:str, userData:dict):
    '''
    Saves given data to a .csv file.
    
    Parameters:
    path (str): Relative path to .csv file with user data.
    userData (dict): Dictionary conatining statistics of a user.
    '''
    
    if not isinstance(userData, dict):
        raise Exception('error while saving user data to csv file: expected dict, received {}'.format(type(userData)))
    
    try:
        with open(path, 'a') as file:
            writer = csv.DictWriter(file, DATA_HEADERS)
            writer.writerow(userData)
    except FileNotFoundError as fnfError:
        print(fnfError)
    except:
        print('Unknown error occured while saving user data to .csv file')

In [4]:
def loadStatFields(path:str):
    '''
    Loads list of statistics for future server requests.
    
    Parameters: 
    path (str): Relative path to file conatining stat fields.
    
    Returns:
    statFields (str): String of statistics fields separated with comma.
    '''
    
    try:
        fieldList = []
        with open(path, 'r') as file:
            content = file.readlines()
            for statField in content:
                fieldList.append(statField.rstrip())
            statFields = ','.join(fieldList)
            return statFields
            
    except FileNotFoundError as fnfError:
        print(fnfError)
    except:
        print('Unknown error occured while loading .txt data from {}'.format(path))

In [5]:
def playerFilter(playerData:dict, minBattles:int):
    '''
    Check if player fulfills minimal requirements to be added to database.
    
    Parameters:
    playerData (dict): Dictionary with player's statistics.
    minBattles (int): Filter players with at least that many battles played.
    
    Returns:
    True if player fullfils all requirements or False if not.
    '''
    
    if playerData['battles'] < minBattles:
        return False
    
    return True

In [6]:
def isValidId(userId:int):
    '''
    Checks if specified user ID is valid.
    
    Parameters:
    user_id (int): ID to be checked.
    
    Returns
    isValid (bool): Returns True if ID is valid and False if it is not.
    '''
    
    if (not isinstance(userId, int) or len(str(userId)) != 9):
        raise Exception('user ID must be an instance of int and must be 9 digits long')
    
    param_dict = {'application_id':APP_ID,
                  'account_id':userId,
                  'fields':['-statistics']}
    
    userRequest = requests.get(API_URL, param_dict)
    serverResponse = userRequest.json()
    
    if serverResponse['status'] != 'ok':
        raise Exception('error: server response status: {}'.format(serverResponse['status']))
        
    isValid = True
    if serverResponse['data'][str(userId)] == None:
        isValid = False
    
    return isValid

In [7]:
def requestUserData(userId:int, statFields:list[str]):
    '''
    Sends API request and returns user data.
    
    Parameters:
    user_id (int): ID of a user whos data is being requested.
    statFields (list[str]): List of statistics names that will be requested from server.
    
    Returns:
    userData (dict): Dictionary containing user data.
    '''
    if not isValidId(userId):
        raise Exception('error: user with this ID does not exist')
    
    param_dict = {'application_id':APP_ID,
                  'account_id':userId,
                  'fields':statFields}
    
    userStatRequest = requests.get(API_URL, param_dict)
    serverResponse = userStatRequest.json()
    
    if serverResponse['status'] != 'ok':
        raise Exception('error: server response status: {}'.format(serverResponse['status']))
    
    userData = serverResponse['data'][str(userId)]['statistics']['all']
    userData.update({'user_id':userId,
                     'nickname':serverResponse['data'][str(userId)]['nickname']})
    return userData

In [10]:
def getUsersData(nUsers:int):
    '''
    Return statistics for given number of random players.
    
    Parameters:
    nUsers (int): Number of users to get data from.
    
    Returns:
    usersList (list): List of dictionaries containing user Data.
    '''
    
    if (nUsers <= 0 or not isinstance(nUsers, int)):
        raise Exception('nUsers must be an integer value greater than 0')
        
    usersList = []
    statFields = loadStatFields('data/stat_fields.txt')
    while len(usersList) < nUsers:
        random_id = random.randint(100000000, 999999999)
        if isValidId(random_id):
            userData = requestUserData(random_id, statFields)
            if playerFilter(userData, 500):
                usersList.append(userData)
                saveRecordToFile('data/user_data.csv', userData)
    
    return usersList

In [17]:
getUsersData(user_count);